### Question 1.a

In [23]:
using JuMP, GLPK

# Number of nodes (1 source + 4 demand days)
m = 5  # Nodes: 1 source and 4 days (Monday to Thursday)

# Define the arcs, upper bounds (capacity), and costs
arcs = [(1,2), (1,3), (1,4), (1,5), (2,3), (3,4), (4,5)]

# Define upper bounds for each arc (production limits and inventory limits)
u = Dict(arcs .=> [700, 700, 700, 700, 1000, 1000, 1000])

# Define costs for each arc (production and inventory costs)
c = Dict(arcs .=> [31, 33, 38, 30, 0.5, 0.5, 0.5])

# Define node surpluses/deficits
b = [1800, -500, -400, -600, -300]  # Total supply at source (1800), and demands for each day

# Create a new model using the GLPK solver
model = Model(GLPK.Optimizer)

# Create variables for flow on each arc
@variable(model, flow[arcs] >= 0)

# Objective: Minimize total cost (sum of production and inventory costs)
@objective(model, Min, sum(c[arc] * flow[arc] for arc in arcs))

# Capacity constraints: Flow on each arc should not exceed the upper bound
for arc in arcs
    @constraint(model, flow[arc] <= u[arc])
end

# Source node constraint (node 1): Production must equal total supply
@constraint(model, sum(flow[arc] for arc in arcs if arc[1] == 1) == b[1])

# Flow balance constraints for nodes 2 to m
for i in 2:m
    @constraint(model, sum(flow[arc] for arc in arcs if arc[2] == i) == sum(flow[arc] for arc in arcs if arc[1] == i) + (-b[i]))
end

# Solve the model
optimize!(model)

# Display results
println("Optimal flow on each arc:")
for arc in arcs
    println("Flow on arc $arc: ", value(flow[arc]))
end

println("Optimal total cost: ", objective_value(model))


Optimal flow on each arc:
Flow on arc (1, 2): 700.0
Flow on arc (1, 3): 700.0
Flow on arc (1, 4): 100.0
Flow on arc (1, 5): 300.0
Flow on arc (2, 3): 200.0
Flow on arc (3, 4): 500.0
Flow on arc (4, 5): 0.0
Optimal total cost: 57950.0


### Question 1b

In [22]:
using JuMP, GLPK

# Number of nodes (1 source + 7 demand days)
m = 8  # Nodes: 1 source and 7 days (Monday to Sunday)

# Define the arcs, upper bounds (capacity), and costs
arcs = [(1,2), (1,3), (1,4), (1,5), (1,6), (1,7), (1,8), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8)]

# Define upper bounds for each arc (production limits and inventory limits)
u = Dict(arcs .=> [700, 700, 700, 700, 600, 600, 600, 1000, 1000, 1000, 1000, 1000, 1000])

# Define costs for each arc (production and inventory costs)
c = Dict(arcs .=> [31, 33, 38, 30, 35, 39, 25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

# Define node surpluses/deficits
b = [3600, -500, -400, -600, -300, -800, -800, -200]  # Total supply at source (3900), and demands for each day

# Create a new model using the GLPK solver
model = Model(GLPK.Optimizer)

# Create variables for flow on each arc
@variable(model, flow[arcs] >= 0)

# Objective: Minimize total cost (sum of production and inventory costs)
@objective(model, Min, 
    sum(c[arc] * flow[arc] for arc in arcs))

# Capacity constraints: Flow on each arc should not exceed the upper bound
for arc in arcs
    @constraint(model, flow[arc] <= u[arc])
end

# Node 1 (source): Production must equal total supply
@constraint(model, sum(flow[arc] for arc in arcs if arc[1] == 1) <= b[1])

# Nodes 2 to 8 (days Monday to Sunday): inflows = outflows + demand
for i in 2:m
    @constraint(model, sum(flow[arc] for arc in arcs if arc[2] == i) == sum(flow[arc] for arc in arcs if arc[1] == i) + (-b[i]))
end

# Solve the model
optimize!(model)

# Display results
println("Optimal flow on each arc:")
for arc in arcs
    println("Flow on arc $arc: ", value(flow[arc]))
end

println("Optimal total cost: ", objective_value(model))


Optimal flow on each arc:
Flow on arc (1, 2): 700.0
Flow on arc (1, 3): 700.0
Flow on arc (1, 4): 100.0
Flow on arc (1, 5): 700.0
Flow on arc (1, 6): 600.0
Flow on arc (1, 7): 600.0
Flow on arc (1, 8): 200.0
Flow on arc (2, 3): 200.0
Flow on arc (3, 4): 500.0
Flow on arc (4, 5): 0.0
Flow on arc (5, 6): 400.0
Flow on arc (6, 7): 200.0
Flow on arc (7, 8): 0.0
Optimal total cost: 119650.0


### 1C

In [28]:
using JuMP, GLPK

# Number of nodes (1 source + 7 demand days)
m = 8  # Nodes: 1 source and 7 days (Monday to Sunday)

# Define the arcs, upper bounds (capacity), and costs
arcs = [(1,2), (1,3), (1,4), (1,5), (1,6), (1,7), (1,8), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8)]

# Define upper bounds for each arc (production limits and inventory limits)
u = Dict(arcs .=> [700, 700, 700, 700, 600, 600, 600, 1000, 1000, 1000, 1000, 1000, 1000])

# Define costs for each arc (production and inventory costs)
c = Dict(arcs .=> [31, 33, 38, 30, 35, 39, 25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

# Define node surpluses/deficits
b = [3600, -500, -400, -600, -300, -800, -800, -200]  # Total supply at source (3600), and demands for each day

# Create a new model using the GLPK solver
model = Model(GLPK.Optimizer)

# Create variables for flow on each arc
@variable(model, flow[arcs] >= 0)

# Create binary variables for each day indicating whether production occurs
@variable(model, x[2:m], Bin)

# Define Big M
M = 10000  # Big-M should be large enough to cover the maximum possible flow

# Objective: Minimize total cost (sum of production, inventory costs, and startup costs)
@objective(model, Min, 
    sum(c[arc] * flow[arc] for arc in arcs) 
    + sum(10000 * x[i] for i in 2:m)  # Adding startup costs linked to binary variables
)

# Capacity constraints: Flow on each arc should not exceed the upper bound
for arc in arcs
    @constraint(model, flow[arc] <= u[arc])
end

# Node 1 (source): Production must equal total supply
@constraint(model, sum(flow[arc] for arc in arcs if arc[1] == 1) <= b[1])

# Nodes 2 to 8 (days Monday to Sunday): inflows = outflows + demand
for i in 2:m
    @constraint(model, sum(flow[arc] for arc in arcs if arc[2] == i) == sum(flow[arc] for arc in arcs if arc[1] == i) + (-b[i]))
end

# Big M to link flow to binary variables: if production occurs on day i, x[i] = 1
for i in 2:m
    @constraint(model, sum(flow[(1,i)] for arc in arcs if arc[2] == i) <= M * x[i] - 1e-4)
end

# Solve the model
optimize!(model)

# Check solver status
status = termination_status(model)
println("Solver status: ", status)

# If infeasible, check the balance between supply and demand
if status == MOI.INFEASIBLE
    println("Model is infeasible. Let's check supply and demand balance.")
    total_supply = b[1]
    total_demand = -sum(b[2:m])
    println("Total supply: $total_supply")
    println("Total demand: $total_demand")
    if total_supply < total_demand
        println("Infeasibility could be caused by insufficient supply to meet demand.")
    else
        println("Supply is sufficient, but there could be other issues.")
    end
else
    # Only print values if the solution is feasible
    if status == MOI.OPTIMAL
        # Display results
        println("Optimal flow on each arc:")
        for arc in arcs
            println("Flow on arc $arc: ", value(flow[arc]))
        end

        # Display the binary variable values (production activity)
        println("Production activity on each day (binary variable):")
        for i in 2:m
            println("Production active on day $i: ", value(x[i]))
        end

        println("Optimal total cost: ", objective_value(model))
    end
end


Solver status: OPTIMAL
Optimal flow on each arc:
Flow on arc (1, 2): 700.0
Flow on arc (1, 3): 700.0
Flow on arc (1, 4): 700.0
Flow on arc (1, 5): 700.0
Flow on arc (1, 6): 600.0
Flow on arc (1, 7): 0.0
Flow on arc (1, 8): 200.0
Flow on arc (2, 3): 200.0
Flow on arc (3, 4): 500.0
Flow on arc (4, 5): 600.0
Flow on arc (5, 6): 1000.0
Flow on arc (6, 7): 800.0
Flow on arc (7, 8): 0.0
Production activity on each day (binary variable):
Production active on day 2: 1.0
Production active on day 3: 1.0
Production active on day 4: 1.0
Production active on day 5: 1.0
Production active on day 6: 1.0
Production active on day 7: 0.0
Production active on day 8: 1.0
Optimal total cost: 179950.0001
